# RAG Post Retrieval Optimization - Reranking featuring Bedrock and llamaindex

Reranking is a crucial post-retrieval process that refines the initial set of retrieved documents to improve the relevance and quality of information used for generating responses. After the initial retrieval step, which typically uses methods like vector similarity search, reranking takes the retrieved documents and re-scores their relevance to the query using more sophisticated techniques. These may include cross-encoders, BERT-based models, or other advanced algorithms that can better capture semantic nuances and context. Based on these scores, the documents are reordered, pushing the most relevant and informative ones to the top, thereby enhancing the accuracy and contextual appropriateness of the final response.

In this lab, we will highlight the usage of Bedrock Reranker LLMs (for example Cohere Rerank 3.5) as a reranker model to improve the relevance of the documents retrieved from Amazon SEC filings. We also utilized the Bedrock Nova Pro model and the Bedrock Titan Text Embedding v2.0 model for natural language processing and document embeddings to build this RAG system. The key steps were: ingesting PDF documents into a vector database index, retrieving top relevant nodes based on a query using semantic search, reranking the retrieved nodes using the fine-tuned transformer model deployed on SageMaker, and synthesizing a final response consolidating the reranked information. See below image for details.

- Vector Database (Faiss / local on the notebook for this demo)
- LLM (Amazon Bedrock - Nova Pro)
- Embeddings Model (Bedrock Titan Text Embedding v2.0)
- ReRanking Model (Cohere Reranker 3.5) running on Bedrock
- Llamaindex for orchestration (ingestion, reranking, retrieval and final response synthesis)
- Datasets (Amazon SEC 10-k statements for year 2022 and 2023 )

## Pre-req
You must run the `[workshop_setup.ipynb]`(../lab00-setup/workshop_setup.ipynb) notebook in `lab00-setup` before starting this lab.

In [ ]:
import warnings
warnings.warn("Warning: if you did not run lab00-setup, please go back and run the lab00 notebook") 

In [ ]:
import os
from pathlib import Path

# Define the config content
config_content = """[profile default]
region = us-west-2
output = json
"""

# Get the path to the .aws directory and config file
home_dir = "/home/sagemaker-user"  # SageMaker specific path
aws_dir = os.path.join(home_dir, ".aws")
config_path = os.path.join(aws_dir, "config")

# Check if config file already exists
if os.path.exists(config_path):
    print(f"AWS config file already exists at {config_path}. No changes made.")
else:
    # Create the .aws directory if it doesn't exist
    os.makedirs(aws_dir, exist_ok=True)
    
    # Create the config file with the content
    with open(config_path, "w") as f:
        f.write(config_content + "\n")
    
    print(f"Created directory {aws_dir} and created AWS config file at {config_path}")


### > Setup
We start by importing necessary llamaindex libraries

In [ ]:
from llama_index.embeddings.bedrock import BedrockEmbedding
from llama_index.core.postprocessor import LLMRerank
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core import Settings

We select Anthropic Claude3 Sonnet as our LLM. For embedding model, we are selecting Amazon Titan Text Embed v2.0. Chunk size is set at 512 for this example.

In [ ]:
import json
from typing import Sequence, List
from llama_index.core.settings import Settings
from llama_index.llms.bedrock_converse import BedrockConverse
from llama_index.embeddings.bedrock import BedrockEmbedding, Models
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
#from llama_index.core.postprocessor.bedrock_rerank import AWSBedrockRerank

profile_name = "default"

# define the LLM
llm = BedrockConverse(
    model="us.amazon.nova-pro-v1:0",
    profile_name=profile_name,
)

# define the embedding model
embed_model = BedrockEmbedding(model = "amazon.titan-embed-text-v2:0")

Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512

from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool
import nest_asyncio
nest_asyncio.apply()

### > Document Ingestion
We ingest and index the data stored in data directory. The amazon folder has SEC-10k files from 2022 and 2023.

In [ ]:
# load data
amazon_secfiles = SimpleDirectoryReader(input_dir="../data/lab03/amazon/").load_data()

# build index
amazon_index = VectorStoreIndex.from_documents(
    amazon_secfiles,
    use_async=True,
)

In [ ]:
query_engine = amazon_index.as_query_engine(similarity_top_k=5)
response = query_engine.query(
    "Describe key business risks for Amazon during covid?",
)

print(response)

In [ ]:
print(response.source_nodes)

# Using Bedrock Rerank LLMs with Llamaindex. We will try with Cohere Reranker3.5

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.postprocessor.bedrock_rerank import AWSBedrockRerank

In [ ]:
reranker = AWSBedrockRerank(
    top_n=3,
    model_id="cohere.rerank-v3-5:0",
    region_name="us-west-2",
)

In [ ]:
query_engine = amazon_index.as_query_engine(
    similarity_top_k=5,
    node_postprocessors=[reranker],
)
response = query_engine.query(
    "Describe key business risks for Amazon during covid?",
)

print(response)

In [ ]:
print(response.source_nodes)